## This script is used for the prepraring dataset for the finetuning DNABERT for SARS-CoV-2 virant clasiification 

In [80]:
import os, tarfile
import glob, time, csv, zipfile, gzip
import pandas as pd
import itertools as it
import sys

In [81]:
os.getcwd()

'/Users/pratik/Box/Datasets/work_dir/SARS_finetuned'

In [82]:
variant_dict = {
    "alpha": 0,
    "beta": 1,
    "delta": 2,
    "gamma": 3,
    "eta":4,
    "iota":5,
    "kappa":6,
    "lambda":7,
    "zeta":8
}

continent_dict={
    "Africa": "AF",
    "Asia": "AS",
    "Europe":"EU",
    "North America": "NA",
    "Oceania": "OC",
    "South America":"SA"
}

In [83]:
variant_dict['alpha']

0

In [84]:
continent_dict['North America']

'NA'

In [85]:
#For Mac
parent_dir_path = "/Users/pratik/Box/Datasets/GISAID_SARS_variant/"
#For Windows
#parent_dir_path = "/home/pratik/Work_directory/SARS_finetuned/GISAID"
os.chdir(parent_dir_path)

### Function for creating non-overlappingg chunks with starting position zero and other mentioned starting position

In [86]:
def create_non_overlap_genome(sequence, chunk_size, index, number_of_genome, fasta_file_name):
    #print(index, number_of_genome)
    chunks = []
    indexes= []
    starting_position_list = [0, 500, 800]
    for position in starting_position_list:
        temp_chunks = [sequence[i:i+chunk_size] for i in range(position, len(sequence), chunk_size)]
        temp_index= [i for i in range(position, len(sequence), chunk_size)]
        
        #print(temp_index[0:10], len(temp_index))
        if(len(temp_chunks[-1])<chunk_size):
            del temp_chunks[-1]
        
        if (len(temp_chunks)<len(temp_index)):
            del temp_index[-1]
        chunks.extend(temp_chunks)
        indexes.extend(temp_index)
    last_chunk = sequence[-chunk_size:]
    chunks.append(last_chunk)
    indexes.append(len(sequence) - chunk_size)
    #print (len(chunks), len(indexes), len(chunks) - len(indexes))
    indexes = [str(index)+"_"+ str(fasta_file_name)+"_"+str(number_of_genome)+"_"+ str(item) for item in indexes]
    #print (indexes)
    return chunks, indexes

### Finetuning data preparation

In [87]:
def finetune_data_preparation(file_lines, label, index, fasta_file_name):
    final_list = []
    final_index= []
    number_of_genome= 0
    for key,group in it.groupby(file_lines,lambda line: line.startswith('>')):
        if not key:
            #count= count+1
            group = list(group)
            stripped_group = list(map(lambda s: s.strip(), group))
            concat_group = ' '.join(stripped_group) 
            concat_group = concat_group.replace(" ","")
            temp_list, temp_list_indexes = create_non_overlap_genome(concat_group, 1000, index, number_of_genome, fasta_file_name)
            number_of_genome= number_of_genome+1
            final_list.extend(temp_list)
            final_index.extend(temp_list_indexes)
    print (len(final_index), len(final_list))
    temp_df= pd.DataFrame({'Index':final_index, 'Sequence':final_list})
    print ("temp", temp_df.shape)
    temp_df = temp_df.drop_duplicates(subset=['Sequence']).reset_index(drop=True)
    print (temp_df.shape)
    #print (temp_df)
    temp_df['Label']= label
    return (temp_df)

## Remove contradict entries and save in a `.csv` file

In [91]:
def remv_contradict_entries(g, directory):
    print (g.shape)
    df1= g.groupby('Sequence')
    df2= df1.filter(lambda x: len(x) == 1).reset_index(drop=True)
    print (df2.groupby('Label').count())
    df2.to_csv("../1000_chunk_size/Finetuned_SARS_"+directory+".csv", index=False)

In [92]:
for directory in os.listdir():
    count =0
    print (directory)
    
    #print(continent_dict[directory])
    #input()
    final_df= pd.DataFrame(columns= ['Index','Sequence','Label' ])
    current_path= os.path.join(parent_dir_path,directory)
    #print (current_path)
    folders = os.listdir(directory)
    for subfolder in folders:
        path = os.path.join(current_path, subfolder)
        print (path)
        files= os.listdir(path)
        for file in files:
            if file.endswith("tar"):
                file_path= os.path.join(path, file)
                contents = tarfile.open(file_path)
                #print(contents.getnames())
                for file_name, file_object in zip(contents.getnames(), contents.getmembers()):
                    if file_name.endswith('fasta'):
                        print(file_name)
                        temp_dir_split = file_name.split('.')
                        fasta_file_name = temp_dir_split[0]
                        f=contents.extractfile(file_object)
                        file_content=f.readlines()
                        file_content_str = [x.decode() for x in file_content]
                        label = variant_dict[subfolder]
                        temp_df = finetune_data_preparation(file_content_str, label, continent_dict[directory], fasta_file_name)
                        final_df = final_df.append(temp_df, ignore_index=True)
                        print ("Final",final_df.shape)
    remv_contradict_entries(final_df, directory)

Africa
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/alpha
1629314973953.sequences.fasta
99805 99805
temp (99805, 2)
(16470, 2)
Final (16470, 3)
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/beta
1629316238802.sequences.fasta
430471 430471
temp (430471, 2)
(54038, 2)
Final (70508, 3)
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/delta
1629316615035.sequences.fasta
91926 91926
temp (91926, 2)
(19308, 2)
Final (89816, 3)
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/eta
1629317297200.sequences.fasta
20001 20001
temp (20001, 2)
(7009, 2)
Final (96825, 3)
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/iota
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/kappa
1629317487078.sequences.fasta
786 786
temp (786, 2)
(646, 2)
Final (97471, 3)
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/lambda
/Users/pratik/Box/Datasets/GISAID_SARS_variant/Africa/zeta
1629316826903.sequences.fasta
873 873
temp (873, 2)
(303, 2)
Final (97774, 3)
(97774, 3)
    

In [90]:
[i for i in range(0, 10, 2)]

[0, 2, 4, 6, 8]